# Imports

In [ ]:
%%capture
!pip install texthero
!pip install --upgrade spacy
!pip install pandasql

In [ ]:
import pandas as pd
import random
from texthero import preprocessing
import texthero as hero
from sklearn import metrics as m
from pandasql import sqldf
import torch
import plotly.express as px
import plotly.graph_objects as go

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
labels = ['Aggressive', 'AgreeBut', 'AgreeToDisagree', 'Alternative', 'Answer',
       'AttackValidity', 'BAD', 'Clarification', 'Complaint', 'Convergence',
       'CounterArgument', 'CriticalQuestion', 'DirectNo', 'DoubleVoicing',
       'Extension', 'Irrelevance', 'Moderation', 'NegTransformation',
       'Nitpicking', 'NoReasonDisagreement', 'Personal', 'Positive',
       'Repetition', 'RephraseAttack', 'RequestClarification', 'Ridicule',
       'Sarcasm', 'Softening', 'Sources', 'ViableTransformation',
       'WQualifiers']

In [ ]:
%%capture
!pip install -U --no-cache-dir gdown --pre
!gdown --id 1FjmTPUBkPB3J9cYUXcMpHikvNWcwsjj8&export=download

In [ ]:
path = r"/content/annotated_trees_101.csv"
df = pd.read_csv(path, index_col=0)

# Analysis for presentation

In [ ]:
temp_df = pd.DataFrame(df[labels].sum())

In [ ]:
# temp_df = temp_df.sort_values(by=[0], ascending=False).reset_index()
temp_df.rename(columns={0:'Num. comments','index':'Label'},inplace=True)
temp_df.columns

Index(['level_0', 'Label', 'Num. comments'], dtype='object')

In [ ]:
fig = px.bar(temp_df[1:], x='Label', y='Num. comments', color="Label")
fig.show()

# Extract Quotes

In [ ]:
def extract_quote(text):
  quote = ""
  while text.find("<quote>") != -1:
    start = text.find("<quote>")
    end = text.find("</quote>")
    quote += f"{text[start+7:end]}\n"
    text = text[0:start] + text[end+8:]
  return [text, quote]

example = "<quote> My </quote> Yes! <quote> My </quote>"
extract_quote(example)

[' Yes! ', ' My \n My \n']

In [ ]:
post_texts = df["text"]
extract = df["text"].apply(extract_quote)
df.insert(loc=5,
          column='txt_no_q',
          value=extract.apply(lambda x: x[0]))
df.insert(loc=6,
          column='quote',
          value=extract.apply(lambda x: x[1]))

# Clean Text

In [ ]:
custom_pipeline = [preprocessing.remove_urls, preprocessing.remove_digits]

In [ ]:
df['clean_text'] = hero.clean(df['text'], custom_pipeline)
df.clean_text = df.clean_text.apply(lambda x: str(x).replace('/', ''))
df.clean_text = df.clean_text.apply(lambda x: str(x).replace('<quote>', ' '))

In [ ]:
df['clean_text_no_q'] = hero.clean(df['txt_no_q'], custom_pipeline)
df['clean_quote'] = hero.clean(df['quote'], custom_pipeline)

# Filter Redundant Comments

In [ ]:
redundant_comments = set(['[removed]', '[deleted]'])
redundant_users = set(['DeltaBot'])

In [ ]:
f_df = df[~df.text.isin(redundant_comments)]
f_df = f_df[~f_df.author.isin(redundant_users)]
f_df['text_len'] = f_df.text.apply(lambda x: len(str(x).split()))

In [ ]:
f_df = f_df[f_df.text_len > 3]

# Add Context

In [ ]:
f_df = f_df.reset_index()

In [ ]:
f_df = f_df.rename(columns={'index': 'child_id'})

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
q = """SELECT df1.*, df2.clean_text as p_text, df2.clean_text_no_q as p_text_no_q, df2.clean_quote as p_quote, df2.child_id as p_context_id, df2.tree_id as p_tree_id
       FROM f_df as df1
       left join f_df as df2 on df2.child_id = df1.parent and df1.tree_id = df2.tree_id;"""

names = pysqldf(q)

In [ ]:
n_df = names[names.parent != -1]

In [ ]:
n_df.to_csv('/content/clean_data.csv', index=False)

# Split text

In [ ]:
tree_ids_dict = dict(n_df.tree_id.value_counts())

In [ ]:
counter = 0
test_tree_ids = []
while counter < 1500:
    key = random.sample(list(tree_ids_dict), 1)
    counter += tree_ids_dict[key[0]]
    test_tree_ids.append(key[0])
train_tree_ids = [x for x in tree_ids_dict if x not in test_tree_ids]

In [ ]:
train_tree_ids = set(train_tree_ids)
test_tree_ids = set(test_tree_ids)

In [ ]:
# test_df.columns

In [ ]:
target_columns = ["clean_text", "clean_text_no_q", "clean_quote", 'p_text', "p_text_no_q", "p_quote"] + labels

In [ ]:
test_df = n_df[n_df.tree_id.isin(test_tree_ids)][target_columns]
train_df = n_df[n_df.tree_id.isin(train_tree_ids)][target_columns]

In [ ]:
test_df.to_csv('/content/test_df.csv', index=False)
train_df.to_csv('/content/train_df.csv', index=False)

In [ ]:
f_df[f_df['text_len'] > 100].shape

(3435, 44)

# output analysis

In [ ]:
import pickle
import numpy as np
from tqdm import tqdm

In [ ]:
_labels = ['Aggressive', 'AgreeBut', 'AgreeToDisagree', 'Alternative', 'Answer',
       'AttackValidity', 'BAD', 'Clarification', 'Complaint', 'Convergence',
       'CounterArgument', 'CriticalQuestion', 'DirectNo', 'DoubleVoicing',
       'Extension', 'Irrelevance', 'Moderation', 'NegTransformation',
       'Nitpicking', 'NoReasonDisagreement', 'Personal', 'Positive',
       'Repetition', 'RephraseAttack', 'RequestClarification', 'Ridicule',
       'Sarcasm', 'Softening', 'Sources', 'ViableTransformation',
       'WQualifiers']

In [ ]:
with open('C:/Users/maorb/Desktop/Papers NLP/prediction', 'rb') as f:
    preds = pickle.load(f)

In [ ]:
with open('C:/Users/maorb/Desktop/Papers NLP/true_labels', 'rb') as f:
    actuals = pickle.load(f)

In [ ]:
labels = []
predictions = []
for batch in range(len(preds)):
    for row in range(len(preds[batch])):
        predictions.append(preds[batch][row])
        labels.append(actuals[batch][row])
        

In [ ]:
predictions

[array([-3.3862908, -3.4953222, -4.3932886, -4.5323887, -4.5918818,
        -3.5485864, -4.257318 , -2.3244789, -2.9479537, -3.906773 ,
        -0.7908967, -3.470831 , -3.3362677, -4.02433  , -3.9304981,
        -3.618158 , -3.5771425, -3.8885908, -4.406884 , -4.5672264,
        -3.4144592, -3.0971785, -4.5032125, -4.416164 , -3.7045724,
        -4.003948 , -3.239275 , -4.0469704, -3.6311445, -4.7807527,
        -3.9583654], dtype=float32),
 array([-3.3621047 , -3.526292  , -4.353895  , -4.5214005 , -4.5325212 ,
        -3.5229757 , -4.242809  , -2.2972457 , -2.9668972 , -3.8749988 ,
        -0.88347113, -3.4679284 , -3.3722684 , -4.04136   , -3.9043362 ,
        -3.6104467 , -3.521925  , -3.8387003 , -4.4729834 , -4.506704  ,
        -3.4593701 , -3.069464  , -4.464937  , -4.3961077 , -3.6003995 ,
        -3.9948988 , -3.22263   , -4.049697  , -3.670036  , -4.750268  ,
        -3.9530206 ], dtype=float32),
 array([-3.4110541, -3.522845 , -4.4585905, -4.5582585, -4.656455 ,
        -3.

In [ ]:
#model_outs = pd.DataFrame(predictions, columns=_labels)
actuals_df = pd.DataFrame(labels, columns=_labels)

In [ ]:
#recall={}
precision={}
thresholds = np.linspace(0.001, 0.07, 100)
for thresh in tqdm(thresholds):
    new_predictions = [(torch.sigmoid(torch.tensor(pred)).numpy() >= thresh).astype(int) for pred in predictions]
    model_outs = pd.DataFrame(new_predictions, columns=_labels)
#     actuals_df = pd.DataFrame(actuals, columns=_labels)
    for col in _labels:
        y_true = actuals_df[col]
        y_pred = model_outs[col]
        #recall.append(m.recall_score(y_true, y_pred))
        precision[col] = precision.get(col, []) + [m.precision_score(y_true, y_pred)]


 34%|███████████████████████████▌                                                     | 34/100 [00:02<00:05, 11.92it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

 36%|█████████████████████████████▏                                                   | 36/100 [00:03<00:05, 11.90it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\

 44%|███████████████████████████████████▋                                             | 44/100 [00:03<00:04, 11.58it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision

 48%|██████████████████████████████████████▉                                          | 48/100 [00:04<00:04, 11.53it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision

 52%|██████████████████████████████████████████                                       | 52/100 [00:04<00:04, 11.43it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision

 56%|█████████████████████████████████████████████▎                                   | 56/100 [00:04<00:03, 11.46it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision

...

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavio

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavio

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavio

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [00:08<00:00, 10.90it/s]C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavio

In [ ]:
m_df = pd.DataFrame(precision)
m_df['threshold'] = thresholds
fig = go.Figure()
for col in m_df.columns[:-1]:
    fig.add_trace(go.Scatter(x=m_df['threshold'],y=m_df[col], mode='lines', name=col))
fig.show()

In [ ]:
x = np.zeros(31)
y = actuals[42][0]

In [ ]:
m.precision_score(y,x)

C:\Users\maorb\Anaconda3\envs\gpu2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



0.0

In [ ]:
!pip install --upgrade nbformat

In [ ]:
labels[442]

In [ ]:
_labels[10]

In [ ]:
from tqdm import tqdm

In [ ]:
accuracy=[]
thresholds = np.linspace(0.05, 0.3, 500)
for thresh in tqdm(thresholds[:10]):
    accuracy_per_preds = []
    for idx, pred in enumerate(predictions):
        pred =  torch.sigmoid(torch.tensor(pred)).numpy()
#         accuracy_per_preds.append(accuracy_score(list(labels[idx]),list((pred >= thresh).astype(int))))
        accuracy_per_preds.append(accuracy_score(list(labels[idx]),list((pred >= thresh).astype(int))))
    accuracy.append(np.mean(accuracy_per_preds))

In [ ]:
accuracy

In [ ]:
sigmoid_preds = []
for idx, pred in enumerate(predictions):
    pred =  torch.sigmoid(torch.tensor(pred)).numpy()
    sigmoid_preds.append(pred)
    accuracy_per_preds.append(accuracy_score(labels[idx], (pred >= 0.05).astype(int)))

In [ ]:
accuracy_per_preds[37]

In [ ]:
sigmoid_preds[37]

In [ ]:
labels[37]

In [ ]:
accuracy_score(list(labels[37]),list((sigmoid_preds[37] >= 0.05).astype(int)))

In [ ]:
accuracy_per_preds[37]